<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=21f4a921623c7b1a85bf2774a5cc610a4f163ac4ab104f2fbf454fb04752b395
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-05 15:40:57
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.40 C
-------------------
Today PnL: -21.23 K (-0.15%)
Current PnL: -18.38 L (-12.36%)
CY Booked + Current PnL: -8.52 L (-5.73%)
-------------------
Total profit:  2.87 L
Total loss:  -21.25 L
-------------------
Total Booked + Current PnL: 18.46 L (15.17%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.06%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 79.09 L (56.62%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.69%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DIXON,18940.15,-28.58,71.0,H-MC,7.43,142840.0,27904.0,8685.0,0.02,24.28,6.08,31.83,81.0,3.21,1.08,48.06,X40N,ATH,IT
4,APOLLOHOSP,8285.00,-14.08,58.0,H-LC,10.04,171699.0,15522.0,10577.0,-1.21,9.94,6.16,16.71,22.0,1.47,1.30,29.11,X40N,BTT,HEALTHCARE
79,TTKPRESTIG,770.00,106.62,67.0,M-SC,6.35,89034.0,-11743.0,11833.0,0.62,-11.65,13.29,0.09,245.0,-0.99,0.67,16.97,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,46.0,M-LC,4.49,213749.0,12270.0,14941.0,-0.35,6.09,6.99,13.51,60.0,0.82,1.61,19.45,XY25,NTT,BANKS
51,NATIONALUM,247.44,-43.66,74.0,H-MC,0.62,112434.0,11930.0,18709.0,2.65,11.87,16.64,30.49,79.0,0.64,0.85,48.12,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIPRULI,790.00,-21.57,36.0,H-MC,2.21,134866.0,-922.0,43670.0,-0.51,-0.68,32.38,31.48,107.0,-0.02,1.02,11.36,X40,ATH,INSURANCE
31,HAVELLS,2069.16,-0.37,60.0,H-MC,1.92,246870.0,-861.0,75913.0,-0.14,-0.35,30.75,30.30,92.0,-0.01,1.86,12.90,X40,ATH,ELECTRICAL
36,ICICIGI,2260.25,-20.81,40.0,H-MC,4.13,136190.0,57.0,31065.0,-1.66,0.04,22.81,22.86,91.0,0.00,1.03,12.66,X40,ATH,INSURANCE
53,PGHH,17973.08,-30.02,58.0,H-MC,4.98,202410.0,1590.0,67180.0,-0.23,0.79,33.19,34.25,80.0,0.02,1.53,6.03,X40,ATH,FMCG
88,WIPRO,420.00,-14.25,41.0,M-LC,6.96,151075.0,130.0,109318.0,-0.53,0.09,72.36,72.51,53.0,0.00,1.14,6.11,XR,NTT,IT


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,-2.37,32.0,H-LC,9.98,222783.0,-19141.0,82407.0,-1.68,-7.91,36.99,26.15,8.0,-0.23,1.68,5.28,X40,ATH,IT
85,VBL,671.64,-16.73,36.0,H-LC,6.95,299637.0,-16205.0,128874.0,-4.06,-5.13,43.01,35.67,5.0,-0.13,2.26,8.01,X40N,ATH,FMCG
45,JIOFIN,387.00,-2.04,39.0,H-LC,13.98,216580.0,-826.0,54318.0,-0.02,-0.38,25.08,24.61,48.0,-0.02,1.64,54.23,XY24,BTT,FINANCE
41,INFY,2275.00,-21.03,39.0,H-LC,10.97,307700.0,-5188.0,176866.0,-1.26,-1.66,57.48,54.87,3.0,-0.03,2.32,4.88,X40,BTT,IT
76,TCS,4998.00,-28.04,41.0,H-LC,14.39,286540.0,-59416.0,183271.0,-1.53,-17.17,63.96,35.80,1.0,-0.32,2.16,1.51,X40,BTT,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-5.78,52.0,H-LC,1.91,233798.0,-12353.0,91555.0,-0.28,-5.02,39.16,32.18,16.0,-0.13,1.77,29.22,X200,ATH,IT
52,NESTLEIND,1377.00,-8.10,63.0,H-LC,2.55,282789.0,17363.0,39421.0,-0.34,6.54,13.94,21.40,11.0,0.44,2.14,13.62,XY25,NTT,FMCG
43,ITC,452.00,-38.34,48.0,H-LC,3.05,199194.0,-944.0,21832.0,-2.06,-0.47,10.96,10.44,4.0,-0.04,1.50,5.07,X40,NTT,FMCG
19,CIPLA,1795.00,-20.12,49.0,H-LC,5.53,212816.0,8316.0,33093.0,-1.55,4.07,15.55,20.25,10.0,0.25,1.61,13.15,X40N,BTT,PHARMA
85,VBL,671.64,-16.73,36.0,H-LC,6.95,299637.0,-16205.0,128874.0,-4.06,-5.13,43.01,35.67,5.0,-0.13,2.26,8.01,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SHALBY,327.00,1018.02,72.0,M-SC,3.28,152079.0,-29064.0,74534.0,11.84,-16.04,49.01,25.10,235.0,-0.39,1.15,21.88,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.00,3981.08,79.0,M-SC,9.62,127145.0,-20223.0,72040.0,5.82,-13.72,56.66,35.16,236.0,-0.28,0.96,29.87,XY24,NTT,AUTO
84,VALIANTORG,838.00,-317.09,51.0,H-SC,9.63,92274.0,-41012.0,125603.0,5.00,-30.77,136.12,63.47,139.0,-0.33,0.70,54.67,XR,NTT,CHEMICALS
51,NATIONALUM,247.44,-43.66,74.0,H-MC,0.62,112434.0,11930.0,18709.0,2.65,11.87,16.64,30.49,79.0,0.64,0.85,48.12,MH,ATH,METALS
86,VOLTAS,1918.49,2.42,64.0,H-MC,3.99,214920.0,23178.0,72858.0,2.15,12.09,33.90,50.08,99.0,0.32,1.62,19.71,XY25,ATH,AC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,VBL,671.64,-16.73,36.0,H-LC,6.95,299637.0,-16205.0,128874.0,-4.06,-5.13,43.01,35.67,5.0,-0.13,2.26,8.01,X40N,ATH,FMCG
68,SIS,528.00,2152.85,40.0,H-SC,5.67,60605.0,-20485.0,29157.0,-3.61,-25.26,48.11,10.69,156.0,-0.70,0.46,22.83,OX40N,NTT,MISC
21,COFFEEDAY,80.00,-41.65,57.0,L-SC,36.65,85555.0,-27994.0,68042.0,-3.59,-24.65,79.53,35.27,268.0,-0.41,0.65,109.40,XR,NTT,HOTELS
71,SURYODAY,240.00,35.96,41.0,H-SC,4.66,96641.0,-53285.0,92959.0,-2.21,-35.54,96.19,26.46,135.0,-0.57,0.73,23.80,XR,NTT,BANKS
67,SIEMENS,7969.85,-3.90,46.0,H-LC,30.44,154455.0,-31640.0,244039.0,-2.07,-17.00,158.00,114.13,15.0,-0.13,1.17,13.74,AR,ATH,ELECTRICAL


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,106.62,67.0,M-SC,6.35,89034.0,-11743.0,11833.0,0.62,-11.65,13.29,0.09,245.0,-0.99,0.67,16.97,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.0,116.52,52.0,M-SC,3.52,142030.0,-32529.0,32567.0,-0.64,-18.63,22.93,0.02,221.0,-1.00,1.07,23.54,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-68.34,54.0,H-SC,3.48,218592.0,-51075.0,87415.0,-0.15,-18.94,39.99,13.47,138.0,-0.58,1.65,10.56,XY24,NTT,PAINTS
80,UJJIVANSFB,60.0,82.03,41.0,H-SC,13.18,114939.0,-27540.0,47056.0,-1.25,-19.33,40.94,13.70,163.0,-0.59,0.87,35.66,OX40N,NTT,BANKS
18,CERA,9475.0,-20.05,45.0,H-SC,2.77,114138.0,-29665.0,56407.0,-0.38,-20.63,49.42,18.60,149.0,-0.53,0.86,25.95,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-28.04,41.0,H-LC,14.39,286540.0,-59416.0,183271.0,-1.53,-17.17,63.96,35.80,1.0,-0.32,2.16,1.51,X40,BTT,IT
41,INFY,2275.00,-21.03,39.0,H-LC,10.97,307700.0,-5188.0,176866.0,-1.26,-1.66,57.48,54.87,3.0,-0.03,2.32,4.88,X40,BTT,IT
43,ITC,452.00,-38.34,48.0,H-LC,3.05,199194.0,-944.0,21832.0,-2.06,-0.47,10.96,10.44,4.0,-0.04,1.50,5.07,X40,NTT,FMCG
85,VBL,671.64,-16.73,36.0,H-LC,6.95,299637.0,-16205.0,128874.0,-4.06,-5.13,43.01,35.67,5.0,-0.13,2.26,8.01,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,44.0,H-LC,13.12,245635.0,-15984.0,135198.0,-0.88,-6.11,55.04,45.57,7.0,-0.12,1.86,4.52,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.09,187705.0,36792.0,20516.0,0.22,24.38,10.93,37.97,93.0,1.79,1.42,55.85,AR,ATH,AUTO
25,DIXON,18940.15,-28.58,71.0,H-MC,7.43,142840.0,27904.0,8685.0,0.02,24.28,6.08,31.83,81.0,3.21,1.08,48.06,X40N,ATH,IT
26,DMART,5201.00,-5.97,69.0,H-LC,15.49,234872.0,50391.0,19988.0,-0.75,27.31,8.51,38.14,38.0,2.52,1.77,40.82,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,12.54,63.0,H-MC,3.37,194350.0,29680.0,38287.0,-0.03,18.02,19.70,41.28,89.0,0.78,1.47,27.45,X40N,ATH,AC
86,VOLTAS,1918.49,2.42,64.0,H-MC,3.99,214920.0,23178.0,72858.0,2.15,12.09,33.90,50.08,99.0,0.32,1.62,19.71,XY25,ATH,AC
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.09,187705.0,36792.0,20516.0,0.22,24.38,10.93,37.97,93.0,1.79,1.42,55.85,AR,ATH,AUTO
23,DABUR,735.00,-2.03,63.0,H-MC,4.42,213213.0,16185.0,73431.0,-1.09,8.21,34.44,45.49,102.0,0.22,1.61,22.33,XY24,BTT,FMCG
17,CAMS,5211.76,-5.61,50.0,H-SC,5.47,107307.0,5303.0,38620.0,1.80,5.20,35.99,43.06,122.0,0.14,0.81,23.20,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.09,187705.0,36792.0,20516.0,0.22,24.38,10.93,37.97,93.0,1.79,1.42,55.85,AR,ATH,AUTO
51,NATIONALUM,247.44,-43.66,74.0,H-MC,0.62,112434.0,11930.0,18709.0,2.65,11.87,16.64,30.49,79.0,0.64,0.85,48.12,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-39.78,61.0,M-SC,4.20,101775.0,10276.0,68474.0,0.56,11.23,67.28,86.07,223.0,0.15,0.77,47.80,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.47,46.0,H-SC,12.22,135420.0,12084.0,121675.0,-0.24,9.80,89.85,108.45,119.0,0.10,1.02,34.34,AR,ATH,MISC
88,WIPRO,420.00,-14.25,41.0,M-LC,6.96,151075.0,130.0,109318.0,-0.53,0.09,72.36,72.51,53.0,0.00,1.14,6.11,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-41.65,57.0,L-SC,36.65,85555.0,-27994.0,68042.0,-3.59,-24.65,79.53,35.27,268.0,-0.41,0.65,109.40,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.17,45.0,H-SC,15.65,89165.0,-11798.0,106329.0,-1.22,-11.69,119.25,93.63,148.0,-0.11,0.67,32.81,SR,ATH,CHEMICALS
50,MASFIN,398.61,-17.80,52.0,H-SC,12.88,93225.0,-4755.0,26355.0,0.03,-4.85,28.27,22.05,152.0,-0.18,0.70,35.51,XR,ATH,FINANCE
51,NATIONALUM,247.44,-43.66,74.0,H-MC,0.62,112434.0,11930.0,18709.0,2.65,11.87,16.64,30.49,79.0,0.64,0.85,48.12,MH,ATH,METALS
80,UJJIVANSFB,60.00,82.03,41.0,H-SC,13.18,114939.0,-27540.0,47056.0,-1.25,-19.33,40.94,13.70,163.0,-0.59,0.87,35.66,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-41.65,57.0,L-SC,36.65,85555.0,-27994.0,68042.0,-3.59,-24.65,79.53,35.27,268.0,-0.41,0.65,109.40,XR,NTT,HOTELS
7,ATULAUTO,844.00,3981.08,79.0,M-SC,9.62,127145.0,-20223.0,72040.0,5.82,-13.72,56.66,35.16,236.0,-0.28,0.96,29.87,XY24,NTT,AUTO
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.09,187705.0,36792.0,20516.0,0.22,24.38,10.93,37.97,93.0,1.79,1.42,55.85,AR,ATH,AUTO
57,RELAXO,1176.00,-31.46,62.0,H-SC,4.96,63642.0,-51896.0,86884.0,-0.22,-44.92,136.52,30.28,136.0,-0.60,0.48,25.12,X40N,NTT,FOOTWEAR
51,NATIONALUM,247.44,-43.66,74.0,H-MC,0.62,112434.0,11930.0,18709.0,2.65,11.87,16.64,30.49,79.0,0.64,0.85,48.12,MH,ATH,METALS


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.75
1,25,43.71
2,50,73.92


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.00
LC    36.11
MC    25.91
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.91
X40      15.59
X40N     14.28
XY25     10.14
AR        9.49
XR        9.06
OX40N     7.45
MH        1.78
X200      1.77
X5K       1.45
SR        1.10
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.82
H-MC    22.89
H-SC    22.59
M-SC    13.85
M-LC     7.19
M-MC     2.69
L-SC     1.56
L-LC     1.10
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.88,-0.33,34.76
IT,13.41,-16.29,73.95
FINANCE,7.42,-22.76,68.02
BANKS,7.37,-16.85,66.46
PAINTS,6.08,-14.58,40.23
MISC,5.86,-18.56,86.53
ELECTRICAL,5.69,-9.32,69.58
AUTO,4.86,-5.11,50.83
AC,3.82,4.33,28.07


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2771556.0,22
AR,1222738.0,10
XR,1095117.0,13
X40,900061.0,10
X40N,542953.0,11
OX40N,472689.0,10
XY25,408564.0,7
SR,197130.0,2
X5K,126200.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2783476.0,27
H-LC,1436884.0,16
M-SC,1411804.0,17
H-MC,1175726.0,17
M-LC,426511.0,5
M-MC,321655.0,2
L-SC,254414.0,3
L-MC,59451.0,1
L-LC,38964.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,934887.0,6
M-SC,XY24,826886.0,7
H-SC,AR,661647.0,5
H-LC,X40,611356.0,5
H-SC,XR,524929.0,6
H-MC,XY24,457758.0,4
H-LC,AR,379237.0,2
M-MC,XY24,321655.0,2
H-LC,X40N,236351.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
